In [1]:
%load_ext autoreload

In [2]:
import os

import numpy as np

from libertem import api
from libertem.udf.sum import SumUDF
from libertem.udf.base import UDF

In [3]:
%autoreload
ctx = api.Context()

distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at: tcp://134.94.162.235:39270
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:34689'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:37162'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:36185'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:32949'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:34021'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:40232'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:37306'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:40180'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:34326'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:39587'
distributed.nanny - INFO -         Start Nanny at: 'tcp://134.94.162.235:41368

In [4]:
ctx.executor.client.scheduler_info()

{'type': 'Scheduler',
 'id': 'Scheduler-01e5c5e4-7ad4-4d68-b9a6-76d31363017b',
 'address': 'tcp://134.94.162.235:39270',
 'services': {},
 'workers': {'tcp://134.94.162.235:33480': {'type': 'Worker',
   'id': 'cpu-0',
   'host': '134.94.162.235',
   'resources': {'CPU': 1, 'compute': 1},
   'local_directory': '/cachedata/users/weber/LiberTEM-uellue/prototypes/cupy/dask-worker-space/worker-nb6cn4mx',
   'name': 'cpu-0',
   'nthreads': 1,
   'memory_limit': 7504577649,
   'last_seen': 1588025394.6791086,
   'services': {},
   'metrics': {'cpu': 0.0,
    'memory': 24162304,
    'time': 1588025394.620996,
    'read_bytes': 4074710.8105906313,
    'write_bytes': 4510371.714867617,
    'num_fds': 23,
    'executing': 0,
    'in_memory': 0,
    'ready': 0,
    'in_flight': 0,
    'bandwidth': {'total': 100000000, 'workers': {}, 'types': {}}},
   'nanny': 'tcp://134.94.162.235:42283'},
  'tcp://134.94.162.235:34852': {'type': 'Worker',
   'id': 'cpu-11',
   'host': '134.94.162.235',
   'resour

In [5]:
def info():
    import os
    return (os.environ.get("LIBERTEM_USE_CPU", None), os.environ.get("LIBERTEM_USE_CUDA", None))

In [6]:
ctx.executor.client.run(info)

{'tcp://134.94.162.235:33480': ('0', None),
 'tcp://134.94.162.235:34852': ('11', None),
 'tcp://134.94.162.235:35665': (None, None),
 'tcp://134.94.162.235:36236': (None, '0'),
 'tcp://134.94.162.235:37668': ('9', None),
 'tcp://134.94.162.235:39402': ('17', None),
 'tcp://134.94.162.235:40159': ('12', None),
 'tcp://134.94.162.235:40762': ('6', None),
 'tcp://134.94.162.235:40786': ('15', None),
 'tcp://134.94.162.235:41827': ('13', None),
 'tcp://134.94.162.235:42464': ('1', None),
 'tcp://134.94.162.235:42939': ('7', None),
 'tcp://134.94.162.235:43378': ('16', None),
 'tcp://134.94.162.235:43664': ('3', None),
 'tcp://134.94.162.235:43678': ('10', None),
 'tcp://134.94.162.235:43785': ('8', None),
 'tcp://134.94.162.235:44757': ('5', None),
 'tcp://134.94.162.235:45390': ('14', None),
 'tcp://134.94.162.235:46155': ('4', None),
 'tcp://134.94.162.235:46712': ('2', None)}

In [7]:
data = np.ones((128, 128, 4, 4))

ds = ctx.load("memory", data=data, num_partitions=128)

In [8]:
class DebugUDF(UDF):
    def get_result_buffers(self):
        return {
            'debug': self.buffer(kind="single", dtype="object"),
            'on_device': self.buffer(kind="sig", dtype=np.float32, where="device"),
            'backend': self.buffer(kind="nav", dtype="object"),
        }
    
    def preprocess(self):
        self.results.debug[0] = dict()
    
    def process_partition(self, partition):
        cpu = os.environ.get("LIBERTEM_USE_CPU", None)
        cuda = os.environ.get("LIBERTEM_USE_CUDA", None)
        self.results.debug[0][self.meta.slice] = (cpu, cuda)
        self.results.on_device[:] += self.xp.sum(partition, axis=0)
        self.results.backend[:] = self.meta.backend
        print(f"meta backend {self.meta.backend}")
        
    def merge(self, dest, src):
        de, sr = dest['debug'][0], src['debug'][0]
        for key, value in sr.items():
            assert key not in de
            de[key] = value
        
        dest['on_device'][:] += src['on_device']
        dest['backend'][:] = src['backend']
            
    def get_backends(self):
        return ('cupy', 'numpy')
        

In [9]:
res = ctx.run_udf(udf=DebugUDF(), dataset=ds)
print(res['debug'].data)
print(res['on_device'].data)
print(res['backend'].data)

[{<Slice origin=(2176, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(4608, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(7040, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(9472, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(11904, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12032, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12160, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12288, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12416, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12544, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12672, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12800, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(12928, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(13056, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(13184, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(13312, 0, 0) shape=(128, 4, 4)>: ('0', None), <Slice origin=(13440, 0, 0

In [11]:
np.any(res['backend'].data == 'cupy')

True